In [10]:
import math
import random
import numpy as np

In [6]:
class iForest:
    def __init__(self,X,t,phi):
        """
        INPUTS:
        X: input data
        t: number of trees
        phi: subsampling size
        
        OUTPUT:
        Forest: a set of t iTrees
        """
        self.size = t
        self.n = phi
        self.forest = []
        self._Train(X,t,phi)
        return
        
    def _Train(self,X,t,phi):
        # l: height limit of iTrees
        l = math.ceil(math.log2(phi))
        for i in range(t):
            # X_prime: subsample of X, used for training the ith iTree
            X_prime = X[np.random.choice(X.shape[0], phi, replace=False), :]
            self.forest.append(iTree(X_prime,0,l))
    
    def predict(self,x):
        h_sum = 0
        for i in range(self.size):
            h_sum += self.forest[i].pathLength(x)
        E = h_sum/self.size
        print('E',E)
        print('c',c(self.n))
        return 2**(-E/c(self.n))
        

In [7]:
class Node:
    def __init__(self,internal=True,left=None,right=None,sAtt=None,sVal=None,size=None):
        self.internal = internal
        self.size = size
        self.left = left
        self.right = right
        self.sAtt = sAtt
        self.sVal = sVal
        return
            

In [8]:
class iTree:
    def __init__(self,X,e,l):
        """
        INPUTS:
        X: input data
        e: current tree height
        l: height limit

        OUTPUT:
        """
        self.X = X
        self.l = l
        self.root = self.build(X,e,l)
        return

    def build(self,X,e,l):
        if e>=l or len(X)<=1:
            return Node(internal=False,size=len(X))
        else:
            q = random.randint(0,len(X[0])-1)
            p = random.uniform(min(X[:,q]),max(X[:,q]))
            Xl = X[X[:,q]<p]
            Xr = X[X[:,q]>=p]
            return Node(internal=True,
                        left=self.build(Xl,e+1,l),
                        right=self.build(Xr,e+1,l),
                        sAtt=q,
                        sVal=p)
    def pathLength(self,x,N=None,e=0):
        """
        INPUTS:
        x: an instance
        N: a Node in the iTree
        e: current path length
        """
        if not N:
            N = self.root
        if not N.internal:
            if N.size>1:
                return e+(N.size>1)*c(N.size)
            else:
                return e
        a = N.sAtt
        if x[a] < N.sVal:
            return self.pathLength(x,N.left,e+1)
        else:
            return self.pathLength(x,N.right,e+1)
        
    def draw(self):
        if len(self.X[0])!=2:
            print('iTree.draw() function only support 2D data')
            return
        plt.figure()
        plt.scatter(self.X[:,0],self.X[:,1])
        limits = [[min(self.X[:,0]),max(self.X[:,0])],[min(self.X[:,1]),max(self.X[:,1])]]
        self._helper(self.root,limits)
                
    def _helper(self,node,limits):
        if not node.internal:
            return
        att = node.sAtt
        val = node.sVal
        point1 = [(1-att)*val+att*limits[1-att][0],att*val+(1-att)*limits[1-att][0]]
        point2 = [(1-att)*val+att*limits[1-att][1],att*val+(1-att)*limits[1-att][1]]
        plt.plot([point1[0],point2[0]],[point1[1],point2[1]])
        limitL = [[limits[0][0],(1-att)*val+att*limits[0][1]],[limits[1][0],(1-att)*limits[1][1]+att*val]]
        limitR = [[(1-att)*val+att*limits[0][0],limits[0][1]],[(1-att)*limits[1][0]+att*val,limits[1][1]]]
        self._helper(node.left,limitL)
        self._helper(node.right,limitR)

In [9]:
def c(n):
    return 2*H(n-1)-(2*(n-1)/n)

def H(i):
    return math.log(i)+0.5772156649